In [1]:
import numpy as np
import pandas as pd 

In [2]:
df = pd.read_csv('/kaggle/input/clean-data-australia-rainfall/rainfall_data_australia.csv', index_col=0)

In [3]:
df.shape

(351587, 17)

In [4]:
df.head()

,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
MinTemp,,,,,,,,,,,,,,,,,
17.9,35.2,0.0,12.0,12.3,48,6.0,20.0,20,13.0,1006.3,1004.4,2,5,26.6,33.4,2,2
17.9,35.2,0.0,12.0,12.3,48,6.0,20.0,20,13.0,1006.3,1004.4,2,5,26.6,33.4,2,2
17.9,35.2,0.0,12.0,12.3,48,6.0,20.0,20,13.0,1006.3,1004.4,2,5,26.6,33.4,2,2
17.9,35.2,0.0,12.0,12.3,48,6.0,20.0,20,13.0,1006.3,1004.4,2,5,26.6,33.4,2,2
17.9,35.2,0.0,12.0,12.3,48,6.0,20.0,20,13.0,1006.3,1004.4,2,5,26.6,33.4,2,2


In [5]:
df.isna().any()

MaxTemp           True
Rainfall         False
Evaporation       True
Sunshine         False
WindGustSpeed    False
WindSpeed9am      True
WindSpeed3pm      True
Humidity9am      False
Humidity3pm       True
Pressure9am       True
Pressure3pm       True
Cloud9am         False
Cloud3pm         False
Temp9am          False
Temp3pm           True
RainToday        False
RainTomorrow     False
dtype: bool

In [6]:
df = df.dropna()

In [7]:
df.isna().any()

MaxTemp          False
Rainfall         False
Evaporation      False
Sunshine         False
WindGustSpeed    False
WindSpeed9am     False
WindSpeed3pm     False
Humidity9am      False
Humidity3pm      False
Pressure9am      False
Pressure3pm      False
Cloud9am         False
Cloud3pm         False
Temp9am          False
Temp3pm          False
RainToday        False
RainTomorrow     False
dtype: bool

In [8]:
df.shape

(349435, 17)